In [2]:
import pandas as pd
import os,sys,inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)

train_navigate = pd.read_csv(parentdir + "/Dataset/navigate.csv", encoding="utf-8")
train_weather = pd.read_csv(parentdir + "/Dataset/weather.csv", encoding="utf-8")
train_schedule= pd.read_csv(parentdir + "/Dataset/schedule.csv", encoding="utf-8")

In [2]:
train_navigate.head()
train_weather.head()
train_schedule.head()

,utterance
0,이번 주 후반에 회의를 위해 회의실 100개를 예약합니다.
1,오늘 저녁식사 시간을 상기시켜 주세요.
2,12일 오후 5시 쯤에 촬영해야 한다는 것을 상기시켜 주세요.
3,제 주치의는 몇 시 며칠에 예약되어 있습니까?
4,회의 일정을 잡아야 합니다.


In [3]:
print(len(train_navigate["utterance"]),len(train_weather["utterance"]),len(train_schedule["utterance"]))

583 896 929


In [4]:
trn = pd.DataFrame(train_navigate["utterance"])
trn["label"]= 1
trn.head()

trw = pd.DataFrame(train_weather["utterance"])
trw["label"]= 2
trw.head()

trs = pd.DataFrame(train_schedule["utterance"])
trs["label"]= 3
trs.head()

,utterance,label
0,이번 주 후반에 회의를 위해 회의실 100개를 예약합니다.,3
1,오늘 저녁식사 시간을 상기시켜 주세요.,3
2,12일 오후 5시 쯤에 촬영해야 한다는 것을 상기시켜 주세요.,3
3,제 주치의는 몇 시 며칠에 예약되어 있습니까?,3
4,회의 일정을 잡아야 합니다.,3


In [5]:
train_data = trn.append(trw)
train_data = train_data.append(trs)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# 전체 데이터를 train : test = 8:2 로 분할
train, test = train_test_split(train_data, test_size=0.20, random_state=42)

In [8]:
train["label"] = pd.Categorical(train["label"])

/home/judy/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
train.groupby("label").count()

,utterance
label,
1,458
2,721
3,747


In [10]:
test.groupby("label").count()

,utterance
label,
1,125
2,175
3,182


In [11]:
from konlpy.tag import Okt
def get_noun(text):
    tokenizer = Okt()
    nouns = Okt().nouns(text)
    return [n for n in nouns]

In [12]:
from konlpy.tag import Okt
def get_stem(text):
    tokenizer = Okt()
    stems = tokenizer.morphs(text)
    return [n for n in stems]

Solution 1 :Pipeline
---

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from konlpy.tag import Okt


tfidf = TfidfVectorizer(max_features = 100, max_df=0.95, min_df=0)
text_clf_svm = Pipeline([('vect', TfidfVectorizer(tokenizer=get_noun, max_features = 100, max_df=0.95, min_df=0)),
                          ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=1000, random_state=42))])

text_clf_svm = text_clf_svm.fit(train["utterance"], train["label"])

from sklearn.externals import joblib
joblib.dump(text_clf_svm, 'model_domain_pipeline.joblib') 

['model_domain_pipeline.joblib']

In [14]:
predicted_svm = text_clf_svm.predict(test["utterance"])
np.mean(predicted_svm == test["label"])

0.9813278008298755

Intent 분류 테스트

In [15]:
def intent(str):
    intent = text_clf_svm.predict([str])
    
    if intent == 1:
        print("Intent : Navigate")
    elif intent == 2:
        print("Intent : Weather")
    else:
        print("Intent : Schedule")

Domain Intent 분류

In [16]:
intent("국민대학교 가는 길 알려줘")

Intent : Navigate


In [17]:
intent("내일 서울에 비가 올까?")

Intent : Weather


In [18]:
intent("금요일 2시 랩미팅 일정 추가해 줘")

Intent : Schedule


Domain Intent 분류

In [19]:
intent("가장 가까운 쇼핑센터가 어디죠?")

Intent : Navigate


In [20]:
intent("오늘 날씨는 어떻습니까?")

Intent : Weather


In [21]:
intent("6일 오전 9시에 영업팀과 회의 일정을 잡으세요.")

Intent : Schedule


In [22]:
intent("오늘 랩미팅 미리 알려줘")

Intent : Schedule


Solution 2 : No Pipeline
---

In [23]:
# https://3months.tistory.com/145

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from konlpy.tag import Okt

vect = TfidfVectorizer(tokenizer=get_noun, max_features = 100, max_df=0.95, min_df=0)
X = vect.fit_transform(train["utterance"])
X.todense()
y = train["label"]


from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=1000, random_state=42)
model.fit(X, y)

SGDClassifier(alpha=0.001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
X_pred = vect.transform(['오늘 서울 하늘 무척이나 맑음'])
y_pred = model.predict(X_pred)

In [25]:
print(y_pred)

[2]


In [26]:
# 테스트 데이터의 스케일을 조정하고 점수를 계산합니다
X_test_scaled = vect.fit_transform(test["utterance"])

predicted_svm = model.predict(X_test_scaled)
np.mean(predicted_svm == test["label"])

0.6348547717842323

In [27]:
# 입력 문장에 대해 Intent 분류
uttr = vect.transform(['오늘 서울 하늘 무척이나 맑음'])
model.predict(uttr)

array([2])

Intent 분류 테스트

In [28]:
def intent(str):
    uttr = vect.transform([str])
    intent = model.predict(uttr)
    
    if intent == 1:
        print("Intent : Navigate")
    elif intent == 2:
        print("Intent : Weather")
    else:
        print("Intent : Schedule")

Domain Intent 분류

In [29]:
intent("국민대학교 가는 길 알려줘")

Intent : Navigate


In [30]:
intent("내일 서울에 비가 올까?")

Intent : Schedule


In [31]:
intent("금요일 2시 랩미팅 일정 추가해 줘")

Intent : Navigate


Domain Intent 분류

In [32]:
intent("가장 가까운 쇼핑센터가 어디죠?")

Intent : Navigate


In [33]:
intent("오늘 날씨는 어떻습니까?")

Intent : Weather


In [34]:
intent("6일 오전 9시에 영업팀과 회의 일정을 잡으세요.")

Intent : Weather


In [35]:
intent("오늘 랩미팅 미리 알려줘")

Intent : Navigate
